In [4]:
%matplotlib notebook
DEFAULT_FIGSIZE = (12, 8)

import sys

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import seaborn as sns
sns.set_style('darkgrid', {'legend.frameon': True})
import pandas as pd

sys.path.append('..')
from antlia import exp2
from antlia import kalman
from antlia import plot_braking as braking
from antlia import dtc
from antlia.plotdf import plotjoint

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = DEFAULT_FIGSIZE
mpl.rcParams['legend.facecolor'] = 'white'

In [5]:
records = exp2.load_records(index=slice(0, 8))

NameError: name 'os' is not defined

In [6]:
# generate Kalman matrices
f, h, F, H = kalman.generate_fhFH(
    constant_velocity=True,
    wheelbase=0.6)

In [7]:
T = 1/125 # bicycle sample rate
q0 = 1
q1 = 0.01
Q = 1*np.diag([
    q0*T**3/6, # [m] x-position
    q0*T**3/6, # [m] y-position
    q1*T**2/2, # [rad/s] yaw angle
    q0*T**2/2, # [m/s] velocity 
    q1*T, # [rad/s] yaw rate
    q0*T, # [m/s^2] acceleration
])

P0 = np.diag([
    0.1,
    0.1,
    0.01,
    1,
    0.1,
    0.2
])

In [345]:
braking_df = pd.read_pickle('trial2_braking_ttc.p.gz')

In [351]:
braking_df[(braking_df['rider id'] == 0) & (braking_df['trial id'] == 0)]['braking starttime'].values[0]

128.377805

In [1]:
import scipy.stats

tol = [0, 0, 0, 0]

linregress_kf = []
linregress_ks = []

def velocity_error(event, result):
    v_measured = event.bicycle.speed
    v_estimated = result.state_estimate[:, 3].squeeze()
    assert v_measured.shape == v_estimated.shape
    
    x = np.square(v_measured - v_estimated).sum()
    return x/x.size

plt.close('all')
for i, r in enumerate(records):
    R = kalman.generate_R(r)
    
    for j, tr in enumerate(r.trials):
        event = tr.event
        #if event.type.value != trial2.EventType.Braking.value:
        #    continue
        #    
        #if (i, j) not in error_set:
        #    continue
        
        z = kalman.generate_measurement(event)
        x0 = kalman.initial_state_estimate(z)
        x0[0] = event.x.max()
        a, b = event.x.max(axis=1)[[0, 20]]
        v = a - b
        if v > 3 and v < 7:
            x0[3] = v 
        assert x0[0] > 20, 'initial x: {:0.3f}'.format(x0[0])
        assert x0[1] > 2.0 and x0[1] < 3.5, 'initial y: {:0.3f}'.format(x0[1])
        assert x0[3] > 1, 'initial velocity: {:0.3f}'.format(x0[3])
        print('rider {} trial {}'.format(i, j))
        print('x0:', x0)
        
        KF = kalman.Kalman(F, H, Q, R, f, h)
        result = KF.estimate(x0, P0, z, tol)
        smooth = KF.smooth_estimate(result)
        ve_kf = velocity_error(event, result)
        ve_ks = velocity_error(event, smooth)
        print('KF v error:', ve_kf)
        print('KS v error:', ve_ks)
        
        #if (ve_ks > 2000) or ((i, j) == (3, 12)):
        if True:
            fig, ax = kalman.plot_kalman_result(result, event, smooth,
                                                wheelbase=0.6,
                                                ax_mask=[1, 0, 0, 1, 0],
                                                figsize=(12, 12))
            # draw instructed speed
            v_inst = exp2.instructed_speed(i, j)
            ax[1].plot(event.bicycle.time[[0, -1]], 2*[v_inst],
                       color='black', linestyle='--',
                       label='instructed speed')
            ax[1].legend()
            plt.show()
        
        #v_kf = result.state_estimate[:, 3].squeeze()
        #v_ks = smooth.state_estimate[:, 3].squeeze()
        #
        #df_ij = braking_df[(braking_df['rider id'] == i) &
        #                   (braking_df['trial id'] == j)]
        #
        #t = event.bicycle.time
        #t0 = df_ij['braking starttime'].values[0]
        #tf = df_ij['braking endtime'].values[0]
        #index = (t >= t0) & (t <= tf)
        #
        ## best-fit line metrics
        #linregress_kf.append(scipy.stats.linregress(
        #    t[index],
        #    v_kf[index]))
        #linregress_ks.append(scipy.stats.linregress(
        #    t[index],
        #    v_ks[index]))

NameError: name 'plt' is not defined

In [364]:
df_kf = pd.DataFrame(linregress_kf,
                     columns=[
                         'linregress slope',
                         'linregress intercept',
                         'linregress r-value',
                         'linregress p-value',
                         'linregress stderr',
                     ])
df_ks = pd.DataFrame(linregress_ks,
                     columns=[
                         'linregress slope',
                         'linregress intercept',
                         'linregress r-value',
                         'linregress p-value',
                         'linregress stderr',
                     ])

In [371]:
slope = []
slope_kf = []
slope_ks = []

for (_, row), (_, row_kf), (_, row_ks) in zip(braking_df.iterrows(),
                                              df_kf.iterrows(),
                                              df_ks.iterrows()):
    a = row['linregress slope']
    b = row_kf['linregress slope']
    c = row_ks['linregress slope']
    
    # skip trial if wheel lockup occurs
    if row['lockup ranges']:
        print('skipping {}-{} due to wheel lockup'.format(
            int(row['rider id']), int(row['trial id'])))
        continue
        
    # skip trial if either fit value exceeds 4 m/s^2
    if abs(a) > 4 or abs(b) > 4 or abs(c) > 4:
        print('skipping {}-{} due to invalid deceleration'.format(
            int(row['rider id']), int(row['trial id'])))
        continue
    slope.append(a)
    slope_kf.append(b)
    slope_ks.append(c)
    
print('\nstats using kalman filter:')
print(scipy.stats.ttest_rel(slope, slope_kf))
print(scipy.stats.linregress(slope, slope_kf))
print('mean: ', np.mean(slope_kf))
    
print('\nstats using kalman smoother:')
print(scipy.stats.ttest_rel(slope, slope_ks))
print(scipy.stats.linregress(slope, slope_ks))
print('mean: ', np.mean(slope_ks))

print('\nmean(from measured): ', np.mean(slope))

skipping 1-5 due to invalid deceleration
skipping 2-0 due to wheel lockup
skipping 2-2 due to wheel lockup
skipping 4-1 due to invalid deceleration

stats using kalman filter:
Ttest_relResult(statistic=1.1028376230902326, pvalue=0.27410282979955575)
LinregressResult(slope=0.90755010021632032, intercept=-0.17397458497520391, rvalue=0.9580272659858079, pvalue=5.5258803233922315e-37, stderr=0.03368441880573874)
mean:  -1.58922365019

stats using kalman smoother:
Ttest_relResult(statistic=-0.47639047971251658, pvalue=0.63536959552947725)
LinregressResult(slope=0.8244311953069674, intercept=-0.25846734033889929, rvalue=0.94238172300863177, pvalue=1.2819044767984043e-32, stderr=0.036300976565419668)
mean:  -1.54409937125

mean(from measured):  -1.55941701166


In [381]:
slope = []
slope_kf = []
slope_ks = []

invalid_set = {
    (1, 3),
    (1, 5),
    (1, 9),
    (1, 15),
    (1, 17),
    (2, 1),
    (2, 2),
    (2, 7),
    (3, 15),
    (3, 16),
    (3, 17),
    (4, 1),
    (4, 2),
    (4, 6),
    (4, 7),
    (4, 12),
    (7, 17)
}
    
skipped = []
for (_, row), (_, row_kf), (_, row_ks) in zip(braking_df.iterrows(),
                                              df_kf.iterrows(),
                                              df_ks.iterrows()):
    a = row['linregress slope']
    b = row_kf['linregress slope']
    c = row_ks['linregress slope']
    
    i = int(row['rider id'])
    j = int(row['trial id'])
    
    # skip invalid trials
    if (i, j) in invalid_set:
        skipped.append('{}-{}'.format(i, j))
        continue
        
    slope.append(a)
    slope_kf.append(b)
    slope_ks.append(c)
    
print('skipping: {}'.format(', '.join(skipped)))

print('\nstats using kalman filter:')
print(scipy.stats.ttest_rel(slope, slope_kf))
print(scipy.stats.linregress(slope, slope_kf))
print('mean: ', np.mean(slope_kf))
    
print('\nstats using kalman smoother:')
print(scipy.stats.ttest_rel(slope, slope_ks))
print(scipy.stats.linregress(slope, slope_ks))
print('mean: ', np.mean(slope_ks))

print('\nmean(from measured): ', np.mean(slope))

skipping: 1-3, 1-5, 1-9, 1-15, 1-17, 2-1, 2-2, 2-7, 3-15, 3-16, 3-17, 4-1, 4-2, 4-6, 4-7, 4-12, 7-17

stats using kalman filter:
Ttest_relResult(statistic=5.2430817581494145, pvalue=2.805992169961336e-06)
LinregressResult(slope=1.0175515458178641, intercept=-0.063005359230778657, rvalue=0.98476883071055832, pvalue=3.4656802501645223e-41, stderr=0.024913918565687185)
mean:  -1.5873600947

stats using kalman smoother:
Ttest_relResult(statistic=2.1799448590265116, pvalue=0.033724338070094728)
LinregressResult(slope=0.91769629251235874, intercept=-0.17362211369339398, rvalue=0.96989415610722118, pvalue=1.42908060436185e-33, stderr=0.031953601252120448)
mean:  -1.54838754446

mean(from measured):  -1.49806144144
